In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some fixed parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 2, 3])              # Boundary condition function coefficients
dim_z = 30                            # Latent space dimension
dtype = torch.float                  # Tensor data type

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [5]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace, ksp)
rhs.set_rhs_stencil(mesh, K)

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs)

In [7]:
trainingData = dta.StokesData(range(256))
trainingData.read_data(['IMG'])
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [8]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [9]:
steps = int(3)
for s in range(steps):
    print('step = ', s)
    batch_samples_pf = torch.multinomial(torch.ones(trainingData.n_samples_in), model.batch_size_N_pf)
#     batch_samples_pc = torch.multinomial(torch.ones(trainingData.n_samples_out), model.batch_size_N_pc)
    model.opt_latent_dist_step()
    model.pf_step(batch_samples_pf)
#     model.pc_step(batch_samples_pc)
    

step =  0
loss_pf =  tensor(11633271., grad_fn=<SubBackward0>)
step =  1
loss_pf =  tensor(11158042., grad_fn=<SubBackward0>)
step =  2
loss_pf =  tensor(10993422., grad_fn=<SubBackward0>)


In [10]:
model.plot_input_reconstruction()
f = plt.gcf()
f.suptitle('Untrained, N = 1184', fontsize=32, y=.7)

Text(0.5, 0.7, 'Untrained, N = 1184')

In [11]:
f.suptitle('N = 256', fontsize=32, y=.7)

Text(0.5, 0.7, 'N = 256')

In [12]:
model.z_mean

tensor([[-0.2122, -0.5728,  0.5989,  ...,  0.4764,  0.2043,  0.1993],
        [-0.0009, -0.0677,  0.1149,  ...,  0.0713,  0.0172, -0.0698],
        [ 0.0506,  0.0618, -0.0435,  ..., -0.1301, -0.0155, -0.0368],
        ...,
        [-0.0671, -0.3347,  0.3175,  ...,  0.1313,  0.1006,  0.1257],
        [ 0.0027, -0.1350,  0.0991,  ...,  0.0767,  0.0137,  0.1300],
        [ 0.0009, -0.1020, -0.0239,  ...,  0.0211, -0.0248, -0.0019]],
       requires_grad=True)

In [13]:
sample = model.pfNet(model.z_mean[0, :])

In [14]:
sample.shape

torch.Size([65536])

In [15]:
plt.figure()

<Figure size 640x480 with 0 Axes>

In [16]:
plt.imshow(torch.reshape(sample, (model.data.img_resolution, model.data.img_resolution)).detach().numpy() > .5,
                     cmap='binary')

In [17]:
plt.figure()
plt.imshow(torch.reshape(sample, (model.data.img_resolution, model.data.img_resolution)).detach().numpy())

In [18]:
plt.figure()
trainingData.plot_microstruct(0)

In [19]:
f = rom.get_autograd_fun()

In [20]:
f(torch.ones(16))

tensor([0.0920, 0.1839, 0.2759, 0.3679, 0.1839, 0.3449, 0.5058, 0.6668, 0.8277,
        0.3679, 0.5978, 0.8277, 1.0577, 1.2876, 0.5518, 0.8507, 1.1496, 1.4485,
        1.7474, 0.7358, 1.1036, 1.4715, 1.8394, 2.2073], dtype=torch.float64)

In [21]:
f

<function ROM.get_autograd_fun.<locals>.AutogradROM.apply>